# Pathway and full outcome models

In [1]:
# import required modules
import numpy as np
import pandas as pd
from math import sqrt
from scipy import stats

from classes.pathway import SSNAP_Pathway
from classes.clinical_outcome import Clinical_outcome

## Import data

Probability distributions of modified Rankin scale (mRS):

In [2]:
mrs_dists = pd.read_csv('data/mrs_dist_probs_cumsum.csv', index_col='Stroke type')

In [3]:
mrs_dists

,0,1,2,3,4,5,6
Stroke type,,,,,,,
pre_stroke_nlvo,0.582881,0.745419,0.848859,0.951082,0.993055,1.000000,1.0
pre_stroke_nlvo_ivt_deaths,0.576469,0.737219,0.839522,0.940620,0.982131,0.989000,1.0
pre_stroke_lvo,0.417894,0.560853,0.679283,0.843494,0.957269,1.000000,1.0
pre_stroke_lvo_ivt_deaths,0.403644,0.541728,0.656119,0.814731,0.924626,0.965900,1.0
pre_stroke_lvo_mt_deaths,0.402850,0.540662,0.654829,0.813128,0.922807,0.964000,1.0
no_treatment_nlvo,0.197144,0.460000,0.580032,0.707768,0.855677,0.917702,1.0
no_effect_nlvo_ivt_deaths,0.197271,0.460000,0.577583,0.702252,0.845244,0.904454,1.0
t0_treatment_nlvo_ivt,0.429808,0.630000,0.738212,0.848427,0.929188,0.956300,1.0
no_treatment_lvo,0.050000,0.129000,0.265000,0.429000,0.676000,0.811000,1.0


Hospital performance parameters:

In [4]:
hospital_performance = pd.read_csv('data/hospital_performance_thrombectomy.csv')

In [5]:
hospital_performance.head(5).T

,0,1,2,3,4
stroke_team,Addenbrooke's Hospital,Basildon University Hospital,Blackpool Victoria Hospital,Broomfield Hospital,Calderdale Royal Hospital
admissions,602.166667,486.5,485.833333,452.166667,634.666667
proportion_of_all_with_ivt,0.149184,0.132237,0.091938,0.104681,0.135504
proportion_of_all_with_mt,0.026571,0.01199,0.01235,0.006635,0.00604
proportion_of_mt_with_ivt,0.520833,0.6,0.444444,0.555556,0.73913
proportion1_of_all_with_onset_known_ivt,0.590645,0.648167,0.44837,0.558791,0.546218
proportion2_of_mask1_with_onset_to_arrival_on_time_ivt,0.679007,0.57241,0.693191,0.605541,0.645673
proportion3_of_mask2_with_arrival_to_scan_on_time_ivt,0.94893,0.958449,0.941501,0.991285,0.946389
proportion4_of_mask3_with_onset_to_scan_on_time_ivt,0.845818,0.888247,0.888628,0.915385,0.929976
proportion5_of_mask4_with_enough_time_to_treat_ivt,1.0,1.0,1.0,1.0,1.0


# Run trials

## Same hospital, multiple trials of the pathway

In [6]:
# Set general model parameters
trials = 4

# Get data for one hospital
hospital_name = hospital_performance.iloc[0]['stroke_team']
hospital_data = hospital_performance.iloc[0]

patient_array = SSNAP_Pathway(hospital_name, hospital_data)
for trial in range(trials):
    patient_array.run_trial()

Show the results for the first few patients in the latest trial:

In [7]:
patient_array.results_dataframe.head(10).T

,0,1,2,3,4,5,6,7,8,9
onset_time_known_bool,True,True,False,False,False,False,False,False,False,True
onset_to_arrival_mins,129.0,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,156.0
onset_to_arrival_on_time_ivt_bool,True,True,False,False,False,False,False,False,False,True
onset_to_arrival_on_time_mt_bool,True,True,False,False,False,False,False,False,False,True
arrival_to_scan_mins,36.0,42.0,25.0,41.0,45.0,60.0,496.0,24.0,49.0,71.0
arrival_to_scan_on_time_ivt_bool,True,True,True,True,True,True,False,True,True,True
arrival_to_scan_on_time_mt_bool,True,True,True,True,True,True,False,True,True,True
onset_to_scan_mins,165.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,227.0
onset_to_scan_on_time_ivt_bool,True,True,False,False,False,False,False,False,False,True
time_left_for_ivt_after_scan_mins,105.0,160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0


Show just the arrival to scan times for each patient in the latest trial:

In [8]:
patient_array.trial['arrival_to_scan_mins'].data

array([ 36.,  42.,  25.,  41.,  45.,  60., 496.,  24.,  49.,  71.,  11.,
        19., 182.,  32., 147., 159.,  63.,  29.,  97.,   9.,  14.,  10.,
       113.,  60., 143.,  49.,  43., 107., 144.,  19.,  84.,  51.,   8.,
        46.,  34.,  52., 118.,   5.,  62.,  49.,  45.,  37.,  70.,  62.,
        40.,  32.,  44.,  43.,  86., 247., 183.,  21.,  27.,  20.,  16.,
        93.,  69., 272., 161., 144.,  50.,  48.,  11.,  43.,   9.,  85.,
        23.,  33.,  55.,  24.,  14.,  23.,  44.,  17.,  30.,  56.,  27.,
        56.,  19.,  19.,  39.,  55.,  29.,  29.,  77.,  11.,  70.,  84.,
        11.,  25.,  38.,  69.,   8.,  13.,  28.,  64.,  28.,  29.,  16.,
        60., 481.,  47.,  76.,  46., 111.,  42.,  82.,  65.,  23.,   6.,
        29.,   9.,  18.,  44.,  33.,  67.,  46.,  68.,  49.,  29.,  36.,
       481.,  16.,  66.,  57.,  56.,  14.,  32.,  49.,  39.,  29.,  55.,
       162.,  75.,  24.,  69.,  20.,  27.,  10., 177.,  61.,  82.,  36.,
        62., 229.,  23.,  80.,  93.,  53.,   6.,  1

Show the hospital performance of each trial:

In [9]:
patient_array.df_performance

,Target,Trial_1,Trial_2,Trial_3,Trial_4
stroke_team,Addenbrooke's Hospital,Addenbrooke's Hospital,Addenbrooke's Hospital,Addenbrooke's Hospital,Addenbrooke's Hospital
admissions,602.166667,602,602,602,602
proportion_of_all_with_ivt,0.149184,0.156146,0.169435,0.177741,0.156146
proportion_of_all_with_mt,0.026571,0.021595,0.023256,0.019934,0.021595
proportion_of_mt_with_ivt,0.520833,0.461538,0.5,0.5,0.461538
proportion1_of_all_with_onset_known_ivt,0.590645,0.576412,0.63289,0.57309,0.578073
proportion2_of_mask1_with_onset_to_arrival_on_time_ivt,0.679007,0.737752,0.708661,0.73913,0.718391
proportion3_of_mask2_with_arrival_to_scan_on_time_ivt,0.94893,0.984375,0.981481,0.984314,0.984
proportion4_of_mask3_with_onset_to_scan_on_time_ivt,0.845818,0.873016,0.845283,0.864542,0.825203
proportion5_of_mask4_with_enough_time_to_treat_ivt,1.0,1.0,1.0,1.0,1.0


## Same hospital, multiple scenarios of the pathway

Make a new pandas dataframe of the hospital performance data for these scenarios:

In [35]:
# Original hospital performance data:
hospital_data_original = hospital_performance.iloc[0]

# Scenario changes to the performance data
scenario_dicts = [
    dict(speed=0, onset=0, benchmark=0),
    dict(speed=0, onset=0, benchmark=1),
    dict(speed=0, onset=1, benchmark=0),
    dict(speed=1, onset=0, benchmark=0),
    dict(speed=0, onset=1, benchmark=1),
    dict(speed=1, onset=0, benchmark=1),
    dict(speed=1, onset=1, benchmark=0),
    dict(speed=1, onset=1, benchmark=1),
    ]
    
trials = 10
for d in scenario_dicts:
    # Create a fresh copy of the original performance data
    hospital_data_scenario = hospital_data_original.copy()
    
    # Keep track of which scenarios are used in here:
    scenarios_list = []
    if d['speed'] == 1:
        # Speed scenario
        # All patients are scanned within 4hrs of arrival
        # (how does this work for the masks picking different times for MT and IVT? Someteims 4hr, sometimes 8hr)
        for key in ['proportion3_of_mask2_with_arrival_to_scan_on_time_ivt',
                    'proportion3_of_mask2_with_arrival_to_scan_on_time_mt']:
            hospital_data_scenario[key] = 1.0  # -------------------------------------------- placeholder value
        scenarios_list.append('speed')
    if d['onset'] == 1:
        # Onset time scenario
        # More patients have their onset time determined
        for key in ['proportion1_of_all_with_onset_known_ivt',
                    'proportion1_of_all_with_onset_known_mt']:
            hospital_data_scenario[key] = 1.0  # -------------------------------------------- placeholder value
        scenarios_list.append('onset')
    if d['benchmark'] == 1:
        # Benchmark scenario
        # The proportion of eligible patients receiving treatment is 
        # in line with the benchmark teams' proportions.
        hospital_data_scenario['proportion6_of_mask5_with_treated_ivt'] = 1.0  # -------------------------------------------- placeholder value
        # hospital_data_scenario['proportion6_of_mask5_with_treated_mt'] = 0.5  # -------------------------------------------- placeholder value
        scenarios_list.append('benchmark')
    
    # Build the scenario name from the options selected above.
    # If none of the options are selected, name this scenario "base".
    scenario_name = ('base' if len(scenarios_list) == 0
                     else ' + '.join(scenarios_list))
    
    hospital_name = hospital_data_scenario['stroke_team']
    hospital_label_here = hospital_name + ': ' + scenario_name
    
    hospital_data_scenario.name = scenario_name

    # Store this data in the results dataframe.
    if scenario_name == 'base':
        df_performance_scenarios = hospital_data_scenario.copy()
    else:
        # Combine the two Series into a single DataFrame:
        df_performance_scenarios = pd.merge(
            df_performance_scenarios, hospital_data_scenario,
            right_index=True, left_index=True)
        
# Transpose the dataframe to match the original hospital_performance 
# format:
df_performance_scenarios = df_performance_scenarios.T

In [36]:
df_performance_scenarios.T

,base,benchmark,onset,speed,onset + benchmark,speed + benchmark,speed + onset,speed + onset + benchmark
stroke_team,Addenbrooke's Hospital,Addenbrooke's Hospital,Addenbrooke's Hospital,Addenbrooke's Hospital,Addenbrooke's Hospital,Addenbrooke's Hospital,Addenbrooke's Hospital,Addenbrooke's Hospital
admissions,602.166667,602.166667,602.166667,602.166667,602.166667,602.166667,602.166667,602.166667
proportion_of_all_with_ivt,0.149184,0.149184,0.149184,0.149184,0.149184,0.149184,0.149184,0.149184
proportion_of_all_with_mt,0.026571,0.026571,0.026571,0.026571,0.026571,0.026571,0.026571,0.026571
proportion_of_mt_with_ivt,0.520833,0.520833,0.520833,0.520833,0.520833,0.520833,0.520833,0.520833
proportion1_of_all_with_onset_known_ivt,0.590645,0.590645,1.0,0.590645,1.0,0.590645,1.0,1.0
proportion2_of_mask1_with_onset_to_arrival_on_time_ivt,0.679007,0.679007,0.679007,0.679007,0.679007,0.679007,0.679007,0.679007
proportion3_of_mask2_with_arrival_to_scan_on_time_ivt,0.94893,0.94893,0.94893,1.0,0.94893,1.0,1.0,1.0
proportion4_of_mask3_with_onset_to_scan_on_time_ivt,0.845818,0.845818,0.845818,0.845818,0.845818,0.845818,0.845818,0.845818
proportion5_of_mask4_with_enough_time_to_treat_ivt,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [37]:

# Set general model parameters
trials = 10

# Set up dataframes.

# Record these measures...
outcome_results_columns = [
    'Thrombolysis_rate_(%)',
    'Thrombectomy_rate_(%)',
    'LVO_IVT_mean_shift',
    'LVO_MT_mean_shift',
    'nLVO_IVT_mean_shift',
    # 'onset_to_needle_mins',
    # 'onset_to_puncture_mins'
]

# ... with these stats...
results_types = [
    '_(median)',
    '_(low_5%)',
    '_(high_95%)',
    '_(mean)',
    '_(stdev)',
    '_(95ci)',
]
# ... and gather all combinations of measure and stat here:
results_columns = [column + ending for column in outcome_results_columns
                   for ending in results_types]

# Also store onset to needle time:
# results_columns += ['Onset_to_needle_(mean)']
results_columns += ['onset_to_needle_mins(mean)']
results_columns += ['onset_to_puncture_mins(mean)']


results_df = pd.DataFrame(columns=results_columns)

# trial dataframe is set up each scenario, but define column names here
trial_columns = [
    'Thrombolysis_rate_(%)',
    'Thrombectomy_rate_(%)',
    'LVO_IVT_mean_shift',
    'LVO_MT_mean_shift',
    'nLVO_IVT_mean_shift',
    'onset_to_needle_mins(mean)',
    'onset_to_puncture_mins(mean)'
    ]

# Iterate through hospitals
scenario_counter = 0
for hospital in df_performance_scenarios.iterrows():
    scenario_counter += 1
    print(f'Scenario {scenario_counter}', end='\r')

    # Get data for one hospital
    hospital_name = hospital[1]['stroke_team'] + ': ' + hospital[0]
    hospital_data = hospital[1]

    # Set up trial results dataframe
    trial_df = pd.DataFrame(columns=trial_columns)

    number_of_patients = int(hospital_data['admissions'])

    patient_array = SSNAP_Pathway(hospital_name, hospital_data)
    for trial in range(trials):

        patient_array.run_trial()
        
        # Initiate the outcome model object:
        clinical_outcome = Clinical_outcome(mrs_dists, number_of_patients)
        # Import patient array data:
        for key in clinical_outcome.trial.keys():
            if key in patient_array.trial.keys():
                clinical_outcome.trial[key].data = patient_array.trial[key].data
        
        # Calculate outcomes:
        results_by_stroke_type, patient_array_outcomes = clinical_outcome.calculate_outcomes()
        
#         # mask = (
#         #     patient_array.patient_array_thrombolysis_conditions_met_bool == 1)
#         # onset_to_needle_mins_masked = \
#         #         patient_array.patient_array_onset_to_needle_mins[mask].mean()

        # Mean treatment times:
        # (if/else to prevent mean of empty slice RunTime warning)
        # Treatment times are only not NaN when the patients received treatment.
        onset_to_needle_mins_mean = (
            np.nanmean(patient_array.trial['onset_to_needle_mins'].data) 
            if np.all(np.isnan(patient_array.trial['onset_to_needle_mins'].data)) == False
            else np.NaN)
        onset_to_puncture_mins_mean = (
            np.nanmean(patient_array.trial['onset_to_puncture_mins'].data) 
            if np.all(np.isnan(patient_array.trial['onset_to_puncture_mins'].data)) == False
            else np.NaN)
    
        # Save scenario results to dataframe
        result = [
            np.mean(patient_array.trial['ivt_chosen_bool'].data)*100.0,
            np.mean(patient_array.trial['mt_chosen_bool'].data)*100.0,
            results_by_stroke_type['lvo_ivt_mean_valid_patients_mean_mrs_shift'],
            results_by_stroke_type['lvo_mt_mean_valid_patients_mean_mrs_shift'],
            results_by_stroke_type['nlvo_ivt_mean_valid_patients_mean_mrs_shift'],
            onset_to_needle_mins_mean,
            onset_to_puncture_mins_mean
        ]
        # print('result', result)
        trial_df.loc[trial] = result
        
        # print(stop, here, please)
        
    trial_result = []
    
    # sometimes these medians etc. are calculated when there's only one or two valid values in the column. Should probably do something about that.
    for column in outcome_results_columns:
        results_here = [
            trial_df[column].median(),
            trial_df[column].quantile(0.05),
            trial_df[column].quantile(0.95),
            trial_df[column].mean(),
            trial_df[column].std(),
            (trial_df[column].mean() -
                stats.norm.interval(0.95, loc=trial_df[column].mean(),
                scale=trial_df[column].std() / sqrt(trials))[0]),
        ]
        trial_result += results_here
    trial_result += [trial_df['onset_to_needle_mins(mean)'].mean()]
    trial_result += [trial_df['onset_to_puncture_mins(mean)'].mean()]
    
    # add scenario results to results dataframe
    results_df.loc[hospital_name] = trial_result
    

# # Apply calibration
# results_df['calibration'] = calibration
# for col in list(results_df):
#     if 'Percent_Thrombolysis' in col or 'Additional_good_outcomes' in col:
#         results_df[col] *= calibration

# round all results to 2 decimal places and return
# results_df = results_df.round(2)
# return (results_df)


/home/anna/miniconda3/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/anna/miniconda3/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/home/anna/miniconda3/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/anna/miniconda3/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [38]:
results_df.head(10).T

,Addenbrooke's Hospital: base,Addenbrooke's Hospital: benchmark,Addenbrooke's Hospital: onset,Addenbrooke's Hospital: speed,Addenbrooke's Hospital: onset + benchmark,Addenbrooke's Hospital: speed + benchmark,Addenbrooke's Hospital: speed + onset,Addenbrooke's Hospital: speed + onset + benchmark
Thrombolysis_rate_(%)_(median),14.534884,33.803987,25.249169,15.531561,59.883721,36.046512,26.079734,60.548173
Thrombolysis_rate_(%)_(low_5%),13.621262,32.799003,23.446844,12.516611,57.117940,33.529900,24.127907,57.325581
Thrombolysis_rate_(%)_(high_95%),17.009967,36.794020,28.214286,18.147841,60.631229,38.039867,27.500000,61.960133
Thrombolysis_rate_(%)_(mean),14.784053,34.302326,25.465116,15.249169,59.418605,36.079734,25.963455,60.232558
Thrombolysis_rate_(%)_(stdev),1.342675,1.591856,1.683223,2.034158,1.405267,1.713461,1.326710,1.846770
Thrombolysis_rate_(%)_(95ci),0.832183,0.986624,1.043253,1.260761,0.870978,1.061995,0.822288,1.144619
Thrombectomy_rate_(%)_(median),2.159468,2.159468,3.654485,2.159468,3.654485,2.159468,3.654485,3.654485
Thrombectomy_rate_(%)_(low_5%),1.993355,2.068106,3.654485,2.159468,3.654485,2.159468,3.654485,3.654485
Thrombectomy_rate_(%)_(high_95%),2.159468,2.250831,3.654485,2.325581,3.654485,2.250831,3.820598,3.820598
Thrombectomy_rate_(%)_(mean),2.109635,2.159468,3.654485,2.192691,3.654485,2.176080,3.704319,3.704319


## All hospitals, multiple trials and outcomes

This needs tidying up

In [11]:

# Set general model parameters
trials = 100

# Set up dataframes.

# Record these measures...
outcome_results_columns = [
    'Thrombolysis_rate_(%)',
    'Thrombectomy_rate_(%)',
    'LVO_IVT_mean_shift',
    'LVO_MT_mean_shift',
    'nLVO_IVT_mean_shift',
    # 'onset_to_needle_mins',
    # 'onset_to_puncture_mins'
]

# ... with these stats...
results_types = [
    '_(median)',
    '_(low_5%)',
    '_(high_95%)',
    '_(mean)',
    '_(stdev)',
    '_(95ci)',
]
# ... and gather all combinations of measure and stat here:
results_columns = [column + ending for column in outcome_results_columns
                   for ending in results_types]

# Also store onset to needle time:
# results_columns += ['Onset_to_needle_(mean)']
results_columns += ['onset_to_needle_mins(mean)']
results_columns += ['onset_to_puncture_mins(mean)']


results_df = pd.DataFrame(columns=results_columns)

# trial dataframe is set up each scenario, but define column names here
trial_columns = [
    'Thrombolysis_rate_(%)',
    'Thrombectomy_rate_(%)',
    'LVO_IVT_mean_shift',
    'LVO_MT_mean_shift',
    'nLVO_IVT_mean_shift',
    'onset_to_needle_mins(mean)',
    'onset_to_puncture_mins(mean)'
    ]

# Iterate through hospitals
scenario_counter = 0
for hospital in hospital_performance.iterrows():
    scenario_counter += 1
    print(f'Scenario {scenario_counter}', end='\r')

    # Get data for one hospital
    hospital_name = hospital[1]['stroke_team']
    hospital_data = hospital[1]

    # Set up trial results dataframe
    trial_df = pd.DataFrame(columns=trial_columns)

    number_of_patients = int(hospital_data['admissions'])

    patient_array = SSNAP_Pathway(hospital_name, hospital_data)
    for trial in range(trials):

        patient_array.run_trial()
        
        # Initiate the outcome model object:
        clinical_outcome = Clinical_outcome(mrs_dists, number_of_patients)
        # Import patient array data:
        for key in clinical_outcome.trial.keys():
            if key in patient_array.trial.keys():
                clinical_outcome.trial[key].data = patient_array.trial[key].data
        
        # Calculate outcomes:
        results_by_stroke_type, patient_array_outcomes = clinical_outcome.calculate_outcomes()
        
#         # mask = (
#         #     patient_array.patient_array_thrombolysis_conditions_met_bool == 1)
#         # onset_to_needle_mins_masked = \
#         #         patient_array.patient_array_onset_to_needle_mins[mask].mean()

        # Mean treatment times:
        # (if/else to prevent mean of empty slice RunTime warning)
        # Treatment times are only not NaN when the patients received treatment.
        onset_to_needle_mins_mean = (
            np.nanmean(patient_array.trial['onset_to_needle_mins'].data) 
            if np.all(np.isnan(patient_array.trial['onset_to_needle_mins'].data)) == False
            else np.NaN)
        onset_to_puncture_mins_mean = (
            np.nanmean(patient_array.trial['onset_to_puncture_mins'].data) 
            if np.all(np.isnan(patient_array.trial['onset_to_puncture_mins'].data)) == False
            else np.NaN)
    
        # Save scenario results to dataframe
        result = [
            np.mean(patient_array.trial['ivt_chosen_bool'].data)*100.0,
            np.mean(patient_array.trial['mt_chosen_bool'].data)*100.0,
            results_by_stroke_type['lvo_ivt_mean_valid_patients_mean_mrs_shift'],
            results_by_stroke_type['lvo_mt_mean_valid_patients_mean_mrs_shift'],
            results_by_stroke_type['nlvo_ivt_mean_valid_patients_mean_mrs_shift'],
            onset_to_needle_mins_mean,
            onset_to_puncture_mins_mean
        ]
        # print('result', result)
        trial_df.loc[trial] = result
        
        # print(stop, here, please)
        
    trial_result = []
    
    # sometimes these medians etc. are calculated when there's only one or two valid values in the column. Should probably do something about that.
    for column in outcome_results_columns:
        results_here = [
            trial_df[column].median(),
            trial_df[column].quantile(0.05),
            trial_df[column].quantile(0.95),
            trial_df[column].mean(),
            trial_df[column].std(),
            (trial_df[column].mean() -
                stats.norm.interval(0.95, loc=trial_df[column].mean(),
                scale=trial_df[column].std() / sqrt(trials))[0]),
        ]
        trial_result += results_here
    trial_result += [trial_df['onset_to_needle_mins(mean)'].mean()]
    trial_result += [trial_df['onset_to_puncture_mins(mean)'].mean()]
    
    # add scenario results to results dataframe
    results_df.loc[hospital_name] = trial_result
    

# # Apply calibration
# results_df['calibration'] = calibration
# for col in list(results_df):
#     if 'Percent_Thrombolysis' in col or 'Additional_good_outcomes' in col:
#         results_df[col] *= calibration

# round all results to 2 decimal places and return
# results_df = results_df.round(2)
# return (results_df)


/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: Run

/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: Run

/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: Run

/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/anna/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: Run

n.b. runtime warning about multiply error a * scale + loc is something to do with the above cell, not either of the pathway or outcome classes it's calling. Maybe that stats.norm.interval() line at the end?

In [12]:
results_df.head(10).T

,Addenbrooke's Hospital,Basildon University Hospital,Blackpool Victoria Hospital,Broomfield Hospital,Calderdale Royal Hospital,Charing Cross Hospital HASU,Colchester General Hospital,Countess of Chester Hospital,Darent Valley Hospital,Derriford Hospital
Thrombolysis_rate_(%)_(median),14.700997,13.580247,9.072165,10.840708,14.195584,13.759690,10.970874,12.861736,11.480363,11.491629
Thrombolysis_rate_(%)_(low_5%),12.126246,11.923868,7.010309,8.407080,12.145110,11.434109,8.932039,10.289389,8.459215,9.581431
Thrombolysis_rate_(%)_(high_95%),17.126246,16.275720,11.134021,13.307522,16.253943,15.901163,13.223301,16.430868,14.501511,13.401826
Thrombolysis_rate_(%)_(mean),14.789037,13.783951,9.125773,10.769912,14.244479,13.676357,10.922330,13.086817,11.413897,11.550989
Thrombolysis_rate_(%)_(stdev),1.483780,1.337487,1.246664,1.490226,1.259576,1.409463,1.375898,1.926281,1.693618,1.138786
Thrombolysis_rate_(%)_(95ci),0.290816,0.262143,0.244342,0.292079,0.246872,0.276250,0.269671,0.377544,0.331943,0.223198
Thrombectomy_rate_(%)_(median),2.159468,1.028807,0.824742,0.442478,0.473186,3.875969,0.582524,1.607717,0.302115,3.044140
Thrombectomy_rate_(%)_(low_5%),1.993355,0.823045,0.618557,0.442478,0.473186,3.682171,0.388350,1.286174,0.302115,2.891933
Thrombectomy_rate_(%)_(high_95%),2.325581,1.028807,0.824742,0.663717,0.630915,4.069767,0.582524,1.607717,0.604230,3.196347
Thrombectomy_rate_(%)_(mean),2.137874,0.997942,0.779381,0.455752,0.534700,3.934109,0.493204,1.517685,0.344411,3.077626


In [13]:
results_df.to_csv('full_outcome_pathway_results_refactored.csv')

Print some info about the final trial: